In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions import normal


In [30]:
def gate_sample(gate = "XOR", batch_size = 1):
   # input = (np.random.rand(N, 2) > 0.5) * 1
    
    values = np.array([[0,0],[0,1], [1,0], [1,1]])
    np.random.shuffle(values)
    if gate == "XOR":
        out = (np.logical_xor(values[:,0], values[:,1]) * 1)
        
    elif gate == "AND":
        out = (np.logical_and(values[:,0], values[:,1]) * 1)
    
    out = np.expand_dims(out, axis=-1)
    return values*2-1, out*2-1

def gate_sample_gen(gate = "XOR"):
    while True:
        sample = gate_sample(gate = gate)
        for (i,o) in zip(sample[0], sample[1]):
            yield i,o

def pretty_print(a, cutoff = 13):
    with np.printoptions(formatter=dict(float=lambda t: "%5.3f" % t), precision=2):
        for i in a:
            for j in i:
                #vector_values = np.abs(j.detach().numpy()).__str__()
                vector_values = j.detach().numpy().__str__()
                if len(vector_values) > cutoff:
                    print(vector_values[:cutoff] + "...]",end="")
                    continue
                
                print(vector_values,end="")
            print("")
            
def insert_io(grid, i, o):
    grid[:, 0][1:3] = i.unsqueeze(-1)
    grid[:, -1][2] = o.unsqueeze(-1)
    return grid

gen = gate_sample_gen(gate = "XOR")

In [59]:
gen = gate_sample_gen(gate = "AND")
next(gen)

(array([-1,  1]), array([-1]))

In [3]:
class Rule(nn.Module):
    def __init__(self, cell_state_size = 7):
        super(Rule, self).__init__()
        self.mutation_distribution = normal.Normal(0, 0.01)
        self.cell_state_size = cell_state_size
        
        self.fc1 = nn.Linear(self.cell_state_size * 9, 5, bias = False)
        self.fc2 = nn.Linear(5, 5, bias = False)
        self.fc3 = nn.Linear(5, self.cell_state_size, bias = False)
        
    def forward(self,x):
        x = self.fc1(x)
        
        x = self.fc2(x)
        x = self.fc3(x)
        x = torch.tanh(x)
        return x#.squeeze(-1)

    #get model parameters as vector
    def get_params(self):
        params = []
        for p in self.parameters():
            view = p.view(p.numel())
            params.append(view)
        params = torch.cat(params, dim=0)
        return params
    
        #turn vector into model parameters
    def set_params(self,data):
        idx = 0
        for p in self.parameters():
            view = data[idx:idx+p.numel()].view(p.shape)
            p.data = view
            idx+=p.numel()
        #return model
    
    def gen_child(self):
        mom = self.get_params()
        
        child = Rule(cell_state_size = self.cell_state_size)
        child.set_params(mom + self.mutation_distribution.sample(mom.shape))
        
        return child

    def pad_cell_grid(self, x):
       # print(x)
        x_padded = torch.stack(
            [F.pad(x[:,:,i],(1,1,1,1), "constant", 0).unsqueeze(-1) for i in range(self.cell_state_size)]
        ,-1).squeeze(-2)
        
        return x_padded
    
    def apply_rule(self, x):
        x = x.unfold(0,3,1).unfold(1,3,1)
        #print(x)
        x = x.reshape([x.shape[0], x.shape[1], x.shape[2]* x.shape[3] * x.shape[4]])
        x = self.forward(x)
        #print(x.shape)
        
        x = self.pad_cell_grid(x)
        
        return x
    
    def build_grid(self, n_layers, layer_width):
        cells = torch.stack(
                [F.pad(torch.tanh(torch.rand(layer_width,n_layers)).float() + 1,(1,1,1,1), "constant", 0).unsqueeze(-1) for _ in range(self.cell_state_size)]
            ,-1).squeeze(-2)
        
        return cells
    
rules = []
for i in range(population_size):
    rules.append(Rule(cell_state_size = cell_state_size))

In [36]:
n_samples = 10
time_per_sample = 15
inputs = 2
outputs = 1
#layers = [4,4,4]
n_layers = 3
layer_size = 4
cell_state_size = 2

# = Rule(cell_state_size = cell_state_size)

population_size = 20



def evaluate_rule(r, gen):
    grid = r.build_grid(5,5)

    reward = 0
    for s in range(n_samples):
        n_in, n_out = next(gen)
        
        for t in range(time_per_sample):
            grid = insert_io(grid, torch.tensor(n_in), torch.tensor(n_out))
            #pretty_print(grid)
            grid = r.apply_rule(grid)
            #print()
            #print()
            n_predicted = grid[3][5][0]
            if t > 5:
                reward += -torch.abs(torch.tensor(n_out) - n_predicted)
                print(n_in, n_out, n_predicted)
        print("####")
        break
    return reward.detach().numpy()


def evaluate_population(rules, gen):
    rewards = np.array([0] * population_size)
    for  idx, r  in enumerate(rules):
        rewards[idx] = (evaluate_rule(r, gen))
        
        
    arg_rewards = np.argsort(rewards)[::-1]
    #print(rewards)
    new_rules = []
    for r in arg_rewards[0:5]:
        for i in range(4):
            new_rules.append(rules[r].gen_child())
            
    return new_rules, rewards[arg_rewards[0]]

gen = gate_sample_gen(gate="XOR")

for e in range(1000):
    rules, best_reward = evaluate_population(rules, gen)
    
    print(f"\r{e} | Best Rewards : {best_reward}   ", end="")
#print(len(new_rules))
    

[ 1 -1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9998, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9998, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9998, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9998, grad_fn=<SelectBackward0>)
[ 1 -1] [1] tensor(0.9998, grad_fn=<SelectBackward0>)
####
[-1 -1] [-1] tensor(-0.9997, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9997, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
####


[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
####
[-1  1] [1] tensor(0.9996, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9995, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9995, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9996, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9996, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[-1  1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
####
[-1 -1] [-1] tensor(-0.9997, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9997, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9998, grad_fn=<SelectBackward0>)


[-1 -1] [-1] tensor(-0.9974, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9974, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9974, grad_fn=<SelectBackward0>)
[-1 -1] [-1] tensor(-0.9974, grad_fn=<SelectBackward0>)
####


KeyboardInterrupt: 

In [65]:
gen = gate_sample_gen(gate="AND")
evaluate_rule(rules[0], gen)

[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9996, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
[1 1] [1] tensor(0.9997, grad_fn=<SelectBackward0>)
####


array([-0.00279504], dtype=float32)

In [233]:
a=F.pad(torch.rand([5,5]),(1,1,1,1), "constant", 0).unsqueeze(-1).repeat([1,1,4])

print(a.shape)
pretty_print(a)
#print(a.unsqueeze(-1))
#print(np.expand_dims(a,- 1))

torch.Size([7, 7, 4])
[0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.840 0.840 ...][0.555 0.555 ...][0.722 0.722 ...][0.462 0.462 ...][0.438 0.438 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.009 0.009 ...][0.288 0.288 ...][0.795 0.795 ...][0.537 0.537 ...][0.391 0.391 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.242 0.242 ...][0.447 0.447 ...][0.687 0.687 ...][0.367 0.367 ...][0.716 0.716 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.721 0.721 ...][0.056 0.056 ...][0.565 0.565 ...][0.448 0.448 ...][0.569 0.569 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.228 0.228 ...][0.282 0.282 ...][0.081 0.081 ...][0.816 0.816 ...][0.994 0.994 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...]


In [28]:
inputs = 2
outputs = 1
#layers = [4,4,4]
n_layers = 3
layer_size = 4
cell_state_size = 2

#cells = torch.zeros([layer_size + 2, n_layers + 1, cell_state_size])
cells = torch.stack(
        [F.pad(torch.arange(25).reshape(5,5).float() + 1,(1,1,1,1), "constant", 0).unsqueeze(-1) for _ in range(cell_state_size)]
    ,-1).squeeze(-2)

#cells = torch.zeros([7,7,4])
print(cells.shape)
pretty_print(cells)

torch.Size([7, 7, 2])
[0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000]
[0.000 0.000][1.000 1.000][2.000 2.000][3.000 3.000][4.000 4.000][5.000 5.000][0.000 0.000]
[0.000 0.000][6.000 6.000][7.000 7.000][8.000 8.000][9.000 9.000][10.000 10.00...][0.000 0.000]
[0.000 0.000][11.000 11.00...][12.000 12.00...][13.000 13.00...][14.000 14.00...][15.000 15.00...][0.000 0.000]
[0.000 0.000][16.000 16.00...][17.000 17.00...][18.000 18.00...][19.000 19.00...][20.000 20.00...][0.000 0.000]
[0.000 0.000][21.000 21.00...][22.000 22.00...][23.000 23.00...][24.000 24.00...][25.000 25.00...][0.000 0.000]
[0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000]


In [29]:
#pretty_print(cells.unfold(0,2,1))
unfolded = cells.unfold(0,3,1).unfold(1,3,1)
unfolded = unfolded.reshape([unfolded.shape[0], unfolded.shape[1], unfolded.shape[2]* unfolded.shape[3] * unfolded.shape[4]])
print(unfolded.shape)

#for i in cells.unfold(0,3,1).unfold(1,3,1):
#    print(i)

torch.Size([5, 5, 18])


In [32]:
r = Rule(cell_state_size = cell_state_size)
#r.set_params(torch.ones(r.get_params().shape))
x = r.forward(unfolded)

x_padded = torch.stack(
        [F.pad(x[:,:,i],(1,1,1,1), "constant", 0).unsqueeze(-1) for i in range(cell_state_size)]
    ,-1).squeeze(-2)

pretty_print(x_padded)

[0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000]
[0.000 0.000][0.323 0.287][0.156 0.553][0.213 0.607][0.269 0.656][0.642 0.380][0.000 0.000]
[0.000 0.000][0.631 0.830][0.282 0.925][0.246 0.958][0.209 0.977][0.752 0.692][0.000 0.000]
[0.000 0.000][0.795 0.993][0.095 0.996][0.056 0.998][0.017 0.999][0.808 0.863][0.000 0.000]
[0.000 0.000][0.891 1.000][0.099 1.000][0.138 1.000][0.176 1.000][0.853 0.942][0.000 0.000]
[0.000 0.000][0.401 1.000][0.435 0.999][0.471 1.000][0.506 1.000][0.346 0.714][0.000 0.000]
[0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000]


In [159]:
torch.stack(
        [F.pad(torch.ones([5,5]),(1,1,1,1), "constant", 0).unsqueeze(-1) for _ in range(cell_state_size)]
    ,-1).squeeze(-2)

In [6]:

#pretty_print(a)

In [105]:
print(a.shape)

torch.Size([7, 7, 4])


In [106]:
print(cells.shape)

torch.Size([4, 6, 1])


In [116]:
a=
pretty_print(a)

[0.000][0.000][0.000][0.000][0.000][0.000][0.000]
[0.000][0.083][0.398][0.856][0.789][0.890][0.000]
[0.000][0.737][0.392][0.719][0.461][0.122][0.000]
[0.000][0.287][0.237][0.742][0.180][0.178][0.000]
[0.000][0.609][0.424][0.511][0.561][0.551][0.000]
[0.000][0.969][0.010][0.980][0.304][0.945][0.000]
[0.000][0.000][0.000][0.000][0.000][0.000][0.000]


In [129]:
pretty_print(torch.stack([F.pad(torch.rand([5,5]),(1,1,1,1), "constant", 0).unsqueeze(-1) for _ in range(3)],-1).squeeze(-2))

[0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.467 0.550 ...][0.877 0.327 ...][0.486 0.906 ...][0.168 0.100 ...][0.558 0.601 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.552 0.056 ...][0.882 0.758 ...][0.749 0.937 ...][0.440 0.158 ...][0.893 0.737 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.201 0.587 ...][0.034 0.385 ...][0.763 0.541 ...][0.352 0.996 ...][0.506 0.457 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.453 0.725 ...][0.035 0.747 ...][0.551 0.582 ...][0.825 0.149 ...][0.022 0.601 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.849 0.853 ...][0.389 0.823 ...][0.113 0.464 ...][0.724 0.112 ...][0.701 0.746 ...][0.000 0.000 ...]
[0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...][0.000 0.000 ...]


In [208]:
torch.arange(25).reshape(5,5) + 1

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]])

In [45]:
mutation = torch.zeros([5,5])
torch.normal(mean=0,std=1,out = mutation)

TypeError: normal() received an invalid combination of arguments - got (mean=int, std=int, out=Tensor, ), but expected one of:
 * (Tensor mean, Tensor std, *, torch.Generator generator, Tensor out)
 * (Tensor mean, float std, *, torch.Generator generator, Tensor out)
 * (float mean, Tensor std, *, torch.Generator generator, Tensor out)
 * (float mean, float std, tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [151]:
def insert_io(grid, i, o):
    grid[:, 0][1:3] = i.unsqueeze(-1)
    grid[:, -1][2] = o.unsqueeze(-1)
    return grid


In [152]:
size = 2
r = Rule(2)
#a = #torch.arange(25).reshape(5,5).unsqueeze(-1).repeat(1,1,2)
#pretty_print(a)
g =r.build_grid(5,5)
#pretty_print(g)
pretty_print(insert_io(g, torch.tensor([100,200]), torch.tensor([300])))

[0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000]
[100.000 100....][1.039 1.791][1.932 1.131][1.971 1.657][1.870 1.964][1.504 1.530][0.000 0.000]
[200.000 200....][1.170 1.448][1.271 1.165][1.178 1.679][1.822 1.451][1.771 1.147][300.000 300....]
[0.000 0.000][1.760 1.644][1.101 1.492][1.145 1.860][1.904 1.737][1.941 1.946][0.000 0.000]
[0.000 0.000][1.767 1.773][1.575 1.493][1.933 1.849][1.555 1.426][1.915 1.672][0.000 0.000]
[0.000 0.000][1.274 1.941][1.635 1.082][1.229 1.672][1.678 1.418][1.093 1.533][0.000 0.000]
[0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000][0.000 0.000]


In [113]:
a[:, 0][1:3] = torch.tensor([100,200])

In [114]:
a

tensor([[  0,   1,   2,   3,   4],
        [100,   6,   7,   8,   9],
        [200,  11,  12,  13,  14],
        [ 15,  16,  17,  18,  19],
        [ 20,  21,  22,  23,  24]])

In [120]:
insert_io(a, torch.tensor([100,200]), torch.tensor([300]))

tensor([[  0,   1,   2,   3,   4],
        [100,   6,   7,   8,   9],
        [200,  11,  12,  13, 300],
        [ 15,  16,  17,  18,  19],
        [ 20,  21,  22,  23,  24]])

In [131]:
t = torch.zeros([5])
t[:] = 5
t

tensor([5., 5., 5., 5., 5.])